# Non-linear regression
### Polynomial Ridge regression
Non-linear regression can be achieved by combining a non-linear feature transformation $\boldsymbol{\phi}(\mathbf{x})$ with linear regression models. The prediction model will become $y=\boldsymbol{\phi}(\mathbf{x})^T\mathbf{w}$. An example is a polynomial feature transformation $\boldsymbol{\phi}(x)=(1,x,...,x^M)$. 

The **Non-linear Ridge regression** is obtained by minimising the loss
$$ F(\mathbf{w})=\frac{1}{2}\sum_{i=1}^N(y_i-\boldsymbol{\phi}(\mathbf{x}_i)^T\mathbf{w})+\frac{\lambda}{2} \mathbf{w}^T\mathbf{w}$$

<img src="pictures/KRRsigma.gif" width = "450" style="float: right;"> 
### Kernel Ridge Regression
The regression model can be alternatively defined using a **dual representation** $\mathbf{a}$ 
$$\hat{y}=\sum_{i=1}^N\kappa(\mathbf{x},\mathbf{x}_i)a_i$$
where $\kappa(\mathbf{x},\mathbf{x}_i)$ is a kernel measuring simmilarity between samples $\mathbf{x}$ and $\mathbf{x}_i$, such as Gaussian Kernel. Predictions using Kernel Ridge regression are evaluated as
$$\hat{y}=\mathbf{k(x)}^T(\mathbf{K}+\lambda\mathbf{I})^{-1}\mathbf{y}$$
where $\mathbf{k(x)}=(\kappa(\mathbf{x},\mathbf{x}_1),...,\kappa(\mathbf{x},\mathbf{x}_N))^T$ is a vector of similarities of the training samples with the new sample $\mathbf{x}$ and 
$$\mathbf{K}=\begin{pmatrix}\kappa(\mathbf{x}_1,\mathbf{x}_1)&\dots&\kappa(\mathbf{x}_1,\mathbf{x}_N)\\\vdots& \ddots & \vdots\\ \kappa(\mathbf{x}_N,\mathbf{x}_1)&\dots&\kappa(\mathbf{x}_N,\mathbf{x}_N) \end{pmatrix}$$
is the matrix of pair-wise similarities between training samples. Kernel Ridge regression model fitted to three datapoints with Gaussian kernel with increasing value of $\sigma$ is shown on the right.

In this notebook we will demonstrate fitting of **Polynomial Ridge** and **Kernel Ridge Regression** to predict the GA from a single feature (volume of cortex) and from volumes of 6 brain structures.

### Load data

The code bellow imports the essential libraries and loads the dataset with 6 brain volumes and creates feature matrix `X` and target vector `y`. Run the code.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
%matplotlib inline 


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

def CreateFeaturesTargets(filename):
    
    df = pd.read_csv(filename,header=None)
    
    # convert from 'DataFrame' to numpy array
    data = df.values

    # Features are in columns one to end
    X = data[:,1:]
    
    # Scale features
    X = StandardScaler().fit_transform(X)

    # Labels are in the column zero
    y = data[:,0]

    # return Features and Labels
    return X, y

X,y = CreateFeaturesTargets('datasets/GA-brain-volumes-6-features.csv')

print('Number of samples is', X.shape[0])
print('Number of features is', X.shape[1])

## Univariate non-linear regression

We will explore univariate non-linear regression to understand behaviour of Polynomial Ridge and Gaussian Kernel Ridge regression models. We predict age at scan from the volume of cortex, the first feature in six brain tissue dataset. 

### Create univariate dataset

First we extract the cortical volumes. Run the code bellow.

In [ ]:
# Extract volume of cortex
X_cortex = X[:,0].reshape(-1,1)

# Print dimensions
print('Number of samples is', X_cortex.shape[0])
print('Number of features is', X_cortex.shape[1])

# Plot the dataset
plt.scatter(X_cortex, y) 
plt.xlabel('Cortical Volume')
plt.ylabel('Age at scan (weeks)')
plt.title('Univariate dataset')

In the next cell you are given functions to calculate cross-validated RMSE and plot a univariate regression model. Look at them and run the cell, you will need these functions later.

In [ ]:
def RMSE_CV(model,X,y):
    scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
    print('Average cross-validated RMSE: {} weeks '.format(round(np.sqrt(-scores.mean()),2)))

def PlotRegressionCurve(model, X, y):
    # Plot the data
    plt.scatter(X, y) 
    plt.xlabel('Volume')
    plt.ylabel('GA')

    # Plot the model
    x = np.linspace(X.min(),X.max(),101) 
    x = x.reshape(-1, 1) 
    y = model.predict(x) 
    plt.plot(x, y, 'r-') 
    plt.ylim([25,46])

### Polynomial Ridge Regression
This code fits, plots and evaluates the polynomial ridge regression model. Note that: 
* The model is a `Pipeline` of `PolynomialFeatures`, `StandardScaler` and `Ridge`. 
* Standard scaler normalises the features after the polynomial transformation and this improves the performance of Ridge regression.
* We exclude the feature 1 from `PolynomialFeatures`, because `Ridge` will create an intercept that is not penalised. This also improves the performance of the model.

__Activity 1:__ Play with the parameters `degree` and `alpha` to see the effect on the curve and performance. Answer the following questions:
* Can you find a setting with the lowest error?
* Set polynomial `degree` to 10 and `alpha` to zero. Is the model overfitted?
* Find the parameter `alpha` to reduce overfitting for `degree` 10. 

__Answer:__ 

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline

# create model
model = Pipeline((
("poly_features", PolynomialFeatures(degree=2, include_bias=False)),
("scaler", StandardScaler()),
("ridge", Ridge(alpha=0)),))

# fit model
model.fit(X_cortex,y)

# evaluate model
RMSE_CV(model,X_cortex,y)

# plot model
PlotRegressionCurve(model, X_cortex, y)

The code bellow performs the grid search to find the best parameters for the Polynomial Ridge Regression model that we defined above. Run it to find the best fit.

In [ ]:
# define parameter grid
parameters = {"poly_features__degree": range(1,15),
             "ridge__alpha":np.logspace(-3,3,100)}

# perform grid search
grid_search = GridSearchCV(model, parameters,cv=5)
grid_search.fit(X_cortex,y)

# calculate best CV RMSE
RMSE_CV(grid_search.best_estimator_,X_cortex,y)
# print best parameters
print('Best degree: ',grid_search.best_estimator_.named_steps["poly_features"].degree)
print('Best alpha: ',round(grid_search.best_estimator_.named_steps["ridge"].alpha,3))

### Kernel Ridge Regression

This code fits, plots and evaluates the kernel ridge regression model `KernelRidge`. Note that: 
* The kernel has been set to Gaussian using parameter `kernel='rbf'`
* Parameter `gamma` represents $\frac{1}{2\sigma}$, where $\sigma$ is the standard deviation of the Gaussian kernel. Note that small values of `gamma` correspond to a large kernel and other way round.
* Parameter `alpha` determines strengths of the regularisation.

__Activity 2:__ Play with the parameters `degree` and `alpha` to see the effect on the curve and performance. Answer the following questions:
* Keep `alpha` fixed to `1e-5` and change values of `gamma` to see the effect of the kernel size on the curve. You can for example try settings `1e-5`, `1e-3`, `1e-1`, `1e1` and `1e3`. Which setting performs the best?
* Set `gamma=1` while keeping `alpha=1e-5`. Is the model overfitted?
* Find the parameter `alpha` to reduce overfitting for `gamma=1e1`.

__Answer:__ 

In [ ]:
from sklearn.kernel_ridge import KernelRidge

# create model
model = KernelRidge(kernel='rbf',gamma=1e0,alpha=1e-5)

# fit model
model.fit(X_cortex,y)

# evaluate model
RMSE_CV(model,X_cortex,y)

# plot model
PlotRegressionCurve(model, X_cortex, y)

The code bellow performs the grid search to find the best parameters for the Kernel Ridge Regression model that we defined above. Run it to find the best fit.

In [ ]:
# Define parameter grid
parameters = {"alpha": np.logspace(-5, 5, 11), 
              "gamma": np.logspace(-5, 5, 11)}

# perform grid search
grid_search = GridSearchCV(model, parameters,cv=5)
grid_search.fit(X_cortex,y)

# calculate best CV RMSE
RMSE_CV(grid_search.best_estimator_,X_cortex,y)
# print best parameters
print('Gamma: {} Alpha: {}'.format(round(grid_search.best_estimator_.gamma,3),round(grid_search.best_estimator_.alpha,3)))

## Exercise 4

In this exercise we will fit multivariate non-linear regression model to the dataset with volumes of 6 brain tissues, calculate cross-validater RMSE and check the bias error in the model. We will compare three models
* Linear Ridge Regression
* Polynomial Ridge Regression
* Kernel Ridge Regression with Gaussian Kernel

### Task 4.1: Linear Ridge Regression

The code bellow tunes Linear Ridge Regression model to the dataset with 6 features, with feature matrix `X` and target vector `y`. Run the code and note the performance. The tuned model is saved in `model_lin`.

In [ ]:
print('Linear Ridge Regression:')

# grid for hyperparameter alpha 
parameters = {"alpha": np.logspace(-3,3,7)}

# create ridge model
model = Ridge()

# perform grid search
grid_search = GridSearchCV(model, parameters,cv=5)
grid_search.fit(X, y)

# remember optimised model
model_lin = grid_search.best_estimator_

# Print optimal alpha
print('Best alpha=', round(model_lin.alpha,3))

# Calculate RMSE_CV
rmse_cv = RMSE_CV(model_lin,X,y)

Your task is now to plot expected vs predicted target values to see whether there is the bias in the linear model. Complete the function `PlotTargets` to do that.

In [ ]:
def PlotTargets(model,X,y):
    # predict targets
    y_pred = None
    # plot expected targets on x axis and predicted targets on y axis
    plt.plot(None,None,'o', label = 'Target values')
    plt.plot([27,45],[27,45],'r', label = '$y=\hat{y}$')
    plt.xlabel('Expected target values')
    plt.ylabel('Predicted target values')
    
PlotTargets(model_lin,X,y)
plt.title('Linear Ridge Regression')

 Does the plot show bias error?

__Answer:__ 

### Task 4.2: Polynomial Ridge Regression

Next, you will tune the polynomial ridge regression, measure its performance and plot the target values to see whether there is still bias.

Complete the code before to tune the model. Note that it is saved in `model_poly`

In [ ]:
# create model
model = Pipeline((
("poly_features", PolynomialFeatures(include_bias=False)),
("scaler", StandardScaler()),
("ridge", Ridge())))

# define parameter grid
parameters = {"poly_features__degree": range(1,5),
             "ridge__alpha":np.logspace(-3,3,7)}

# perform grid search
grid_search = None


# remember optimised model
model_poly = grid_search.best_estimator_

Complete the code bellow to print the optimal parameters and evaluate performance. 

In [ ]:
print('Polynomial Ridge Regression:')

# print optimal parameters
print('Best degree: ', model_poly.named_steps["poly_features"].degree)
print('Best alpha: ',round(None,3))

# Calculate CV RMSE
RMSE_CV(model_poly,X,y)

Is the performance better than for Linear Ridge?

__Answer:__ 

Plot the expected vs predicted target values using the function `PlotTargets` that you created.

In [ ]:
# Plot

plt.title('Polynomial Ridge Regression')

Is there less bias than for the linear model?

__Answer:__ 

### Task 4.3: Kernel Ridge Regression

Finally, you will tune the Gaussian Kernel Ridge Regression, measure its performance and plot the target values to see whether the bias error is further reduced.

Complete the code before to tune the model. Note that it is saved in `model_kernel`

In [ ]:
# create model
model = None

# define parameter grid
parameters = {"alpha": np.logspace(-5, 5, 11), 
              "gamma": np.logspace(-5, 5, 21)}

# perform grid search
grid_search = None


# remember optimised model
model_kernel = None

Complete the code bellow to print the optimal parameters and evaluate performance. 

In [ ]:
print('Kernel Ridge Regression:')

# print optimal parameters
print('Best gamma: ', round(None,3))
print('Best alpha: ',round(None,5))

# Calculate CV RMSE


Is the performance better than for Polynomial Ridge?

__Answer:__ 

Plot the expected vs predicted target values using the function `PlotTargets` that you created.

In [ ]:
# Plot

plt.title('Kernel Ridge Regression')

Is there less bias than for the polynomial model?

__Answer:__ 